In [1]:
# A simple DTW Notebook for string matching and error rate evaluation

In [2]:
# install pyspch if needed
try:
  import pyspch
except:
  ! pip install git+https://github.com/compi1234/pyspch.git

# do all the imports
%matplotlib inline

import sys, os
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pyspch import dtw as dtw

### 1. lev_distance() 
The lev_distance() routine computes the symmetric edit distance with equal costs for Deletions, Insertions and Deletions

In [4]:
print(dtw.lev_distance("word","words"))
print(dtw.lev_distance("word","wods"))
print(dtw.lev_distance("words","word"))

1.0
2.0
1.0


### 2. edit_distance()
The edit_distance() routine computes the weighted edit distance.
Normally the weighted edit distance is NOT symmetric and therefore we make explicit
the distinction between a hyp(othesis) and ref(erence).  In a trellis we show the hypothesis on the x-axis and the reference on the y-axis

If you want to see the internals (trellis and matching operations), you can specify the "Verbose" keyword

In [5]:
obs = "donysos"  #"d o n y s o   s"
ref = "doteos"     #"d o t e o s "
dist,align,cts,trellis = dtw.edit_distance(x=list(obs), y=list(ref),Verbose=True)

Edit Distance:  3.0
[[0. 1. 2. 3. 4. 5. 6. 7.]
 [1. 0. 1. 2. 3. 4. 5. 6.]
 [2. 1. 0. 1. 2. 3. 4. 5.]
 [3. 2. 1. 1. 2. 3. 4. 5.]
 [4. 3. 2. 2. 2. 3. 4. 5.]
 [5. 4. 3. 3. 3. 3. 3. 4.]
 [6. 5. 4. 4. 4. 3. 4. 3.]]
[['Q' 'I' 'I' 'I' 'I' 'I' 'I' 'I']
 ['D' 'M' 'I' 'I' 'I' 'I' 'I' 'I']
 ['D' 'D' 'M' 'I' 'I' 'I' 'M' 'I']
 ['D' 'D' 'D' 'S' 'S' 'S' 'S' 'S']
 ['D' 'D' 'D' 'S' 'S' 'S' 'S' 'S']
 ['D' 'D' 'M' 'S' 'S' 'S' 'M' 'I']
 ['D' 'D' 'D' 'S' 'S' 'M' 'S' 'M']]
[(0, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5)]
   x  y  O
0  d  d  M
1  o  o  M
2  n  _  I
3  y  t  S
4  s  e  S
5  o  o  M
6  s  s  M
Number of Words:  6
Substitutions/Insertions/Deletions:  2 1 0


In [6]:
display(align)
display(trellis)

,0,1,2,3,4,5,6
x,d,o,n,y,s,o,s
y,d,o,_,t,e,o,s
O,M,M,I,S,S,M,M


,#,d,o,n,y,s,o,s
#,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
d,1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0
o,2.0,1.0,0.0,1.0,2.0,3.0,4.0,5.0
t,3.0,2.0,1.0,1.0,2.0,3.0,4.0,5.0
e,4.0,3.0,2.0,2.0,2.0,3.0,4.0,5.0
o,5.0,4.0,3.0,3.0,3.0,3.0,3.0,4.0
s,6.0,5.0,4.0,4.0,4.0,3.0,4.0,3.0


### Edit distance for scoring the output of a speech recognizer

The edit_distance() routine can perfectly be used to score the output of a speech recognizer
against a given reference.  
We define a simple helper routine to score a number of sentence:
- tokenize (split a textual reference/input into words for the DTW routine)
- run the dtw.edit_distance() module
- aggregate the results over the corpus

In [7]:
def score_corpus(corpus, Verbose = False, Display = True):
    Nsub = 0
    Nins = 0
    Ndel = 0
    Ntot = 0
    for [reference,result] in corpus:
        ref = dtw.tokenizer(reference)
        hyp = dtw.tokenizer(result)
        _, align, cts, _ = dtw.edit_distance(hyp,ref)
        
        if(Verbose):
            print("Input(reference):",ref)
            print("Output(test):    ",hyp)
            display(align)
        
        Nsub += cts[0]
        Nins += cts[1]
        Ndel += cts[2]
        Ntot += cts[3]

    print("\n ++ CORPUS RESULTS ++ ")
    print("\n#S=%d, #I=%d, #D=%d for %d tokens" % (Nsub,Nins,Ndel,Ntot) )
    print("Error Rate: %5.2f%%" % (100.*(Nsub+Nins+Ndel)/Ntot)  )

In [8]:
ref1="fauchelevent limped along behind the horse in a very contented frame of mind "
obs1 = "lochleven limped along behind the heard in very contented frame of mind"

ref2= " he would have loved to be king in such a non nonsense paradise "
obs2=" he had loved the king in a no sense paradigm"

ref3 = "do you know the names of the seven dwarfs in Disney’s Snow White movie ?"
obs3 =  "do you know the names of the seven warfs in the sneaze now white movie ?"
d2,_,cts2,_ = dtw.edit_distance(x=obs2.split(), y=ref2.split())
d3,_,cts3,_ = dtw.edit_distance(x=obs3.split(), y=ref3.split())
d1,_,cts1,_ = dtw.edit_distance(x=obs1.split(), y=ref1.split())
print("Sentence by Sentence Results")
print(d1,d2,d3)
print(cts1,cts2,cts3)

Sentence by Sentence Results
3.0 8.0 5.0
(2, 0, 1, 13, 23.076923076923077) (5, 0, 3, 13, 61.53846153846154) (4, 1, 0, 15, 33.333333333333336)


In [9]:
# corpus scoring routine
corpus = [ [ref1,obs1], [ref2,obs2], [ref3,obs3] ]
score_corpus(corpus, Verbose = True)

Input(reference): ['fauchelevent', 'limped', 'along', 'behind', 'the', 'horse', 'in', 'a', 'very', 'contented', 'frame', 'of', 'mind']
Output(test):     ['lochleven', 'limped', 'along', 'behind', 'the', 'heard', 'in', 'very', 'contented', 'frame', 'of', 'mind']


,0,1,2,3,4,5,6,7,8,9,10,11,12
x,lochleven,limped,along,behind,the,heard,in,_,very,contented,frame,of,mind
y,fauchelevent,limped,along,behind,the,horse,in,a,very,contented,frame,of,mind
O,S,M,M,M,M,S,M,D,M,M,M,M,M


Input(reference): ['he', 'would', 'have', 'loved', 'to', 'be', 'king', 'in', 'such', 'a', 'non', 'nonsense', 'paradise']
Output(test):     ['he', 'had', 'loved', 'the', 'king', 'in', 'a', 'no', 'sense', 'paradigm']


,0,1,2,3,4,5,6,7,8,9,10,11,12
x,he,_,had,loved,_,the,king,in,_,a,no,sense,paradigm
y,he,would,have,loved,to,be,king,in,such,a,non,nonsense,paradise
O,M,D,S,M,D,S,M,M,D,M,S,S,S


Input(reference): ['do', 'you', 'know', 'the', 'names', 'of', 'the', 'seven', 'dwarfs', 'in', 'Disney’s', 'Snow', 'White', 'movie', '?']
Output(test):     ['do', 'you', 'know', 'the', 'names', 'of', 'the', 'seven', 'warfs', 'in', 'the', 'sneaze', 'now', 'white', 'movie', '?']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
x,do,you,know,the,names,of,the,seven,warfs,in,the,sneaze,now,white,movie,?
y,do,you,know,the,names,of,the,seven,dwarfs,in,_,Disney’s,Snow,White,movie,?
O,M,M,M,M,M,M,M,M,S,M,I,S,S,S,M,M



 ++ CORPUS RESULTS ++ 

#S=11, #I=1, #D=4 for 41 tokens
Error Rate: 39.02%
